In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import nltk
import string
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from tqdm import tqdm

In [54]:
comments=pd.read_csv('dec_reviews.csv.gz')
translated=pd.read_csv('final_december.csv')

In [55]:
comments.dropna(inplace=True)

In [56]:
comments.reset_index(inplace=True)

In [59]:
translated.head()

,translated
0,Our stay of three nights.\nWe appreciate The a...
1,Great experience.
2,I've stayed with my friend at the Midtown Cast...
3,"We've been staying here for about 9 nights, en..."
4,We had a wonderful stay at Jennifer's charming...


In [58]:
translated.rename(columns={"comments": "translated"},inplace=True)

In [60]:
print(comments.shape)
print(translated.shape)

(1254650, 7)
(1254646, 1)


In [61]:
frame=[comments,translated]
final_df=pd.concat(frame,axis=1)

In [62]:
final_df.shape

(1254650, 8)

In [63]:
final_df.isnull().sum()

index            0
listing_id       0
id               0
date             0
reviewer_id      0
reviewer_name    0
comments         0
translated       4
dtype: int64

In [64]:
final_df.dropna(inplace=True)

In [65]:
final_df.head()

,index,listing_id,id,date,reviewer_id,reviewer_name,comments,translated
0,0,2595,17857,2009-11-21,50679,Jean,Notre séjour de trois nuits.\r\nNous avons app...,Our stay of three nights.\nWe appreciate The a...
1,1,2595,19176,2009-12-05,53267,Cate,Great experience.,Great experience.
2,2,2595,19760,2009-12-10,38960,Anita,I've stayed with my friend at the Midtown Cast...,I've stayed with my friend at the Midtown Cast...
3,3,2595,34320,2010-04-09,71130,Kai-Uwe,"We've been staying here for about 9 nights, en...","We've been staying here for about 9 nights, en..."
4,4,2595,46312,2010-05-25,117113,Alicia,We had a wonderful stay at Jennifer's charming...,We had a wonderful stay at Jennifer's charming...


In [66]:
final_df.comments

0          Notre séjour de trois nuits.\r\nNous avons app...
1                                          Great experience.
2          I've stayed with my friend at the Midtown Cast...
3          We've been staying here for about 9 nights, en...
4          We had a wonderful stay at Jennifer's charming...
                                 ...                        
1254641    I enjoyed my stay in the private room, very co...
1254642    The host canceled this reservation 24 days bef...
1254643    The host canceled this reservation 25 days bef...
1254644    The host canceled this reservation 11 days bef...
1254645    The host canceled this reservation 27 days bef...
Name: comments, Length: 1254646, dtype: object

In [67]:
import re
import numpy as np

In [68]:
import gensim

In [69]:
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [70]:
import spacy

In [71]:
# for plotting
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

In [72]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ikhyvicky/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [73]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [77]:
# Convert email body to list
data = final_df.translated.values.tolist()

In [78]:
# tokenize - break down each sentence into a list of words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [79]:
data_words=list(sent_to_words(data))

In [80]:
print(data_words[3])

['we', 've', 'been', 'staying', 'here', 'for', 'about', 'nights', 'enjoying', 'to', 'be', 'in', 'the', 'center', 'of', 'the', 'city', 'that', 'never', 'sleeps', 'short', 'ways', 'to', 'everywhere', 'in', 'manhattan', 'by', 'subway', 'or', 'by', 'walk', 'midtown', 'castle', 'is', 'beauftiful', 'and', 'tastful', 'place', 'jennifer', 'and', 'tori', 'relaxed', 'and', 'friendly', 'hosts', 'thats', 'why', 'we', 'the', 'three', 'berliners', 'recommand', 'that', 'place', 'good', 'to', 'have', 'wifi', 'and', 'little', 'kitchen', 'too']


In [81]:
from gensim.models.phrases import Phrases, Phraser

In [82]:
# Build the bigram and trigram models
bigram = Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[data_words], threshold=100)

In [83]:
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

In [84]:
print(trigram_mod[bigram_mod[data_words[35]]])

['the', 'location', 'made', 'getting', 'to', 'time', 'square', 'easily', 'accessible', 'apartment', 'was', 'quiet', 'and', 'no', 'disturbances', 'from', 'neighbors', 'at', 'all']


In [85]:
#remove stop_words, make bigrams and lemmatize
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [86]:
data_words_nostops = remove_stopwords(data_words)

In [87]:
data_words_bigrams = make_bigrams(data_words_nostops)

In [88]:
import en_core_web_sm

In [89]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = en_core_web_sm.load(disable=['parser', 'ner'])

In [90]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [91]:
import pickle

In [92]:
f=open("./model/data_lemmatized_december.pkl",'wb')
pickle.dump(data_lemmatized,f)
f.close()

In [93]:
len(data_lemmatized)

1254646

In [94]:
#create a dictionary and corpus for (LDA topic modeling)

#creating the dictionary

idtoword=corpora.Dictionary(data_lemmatized)
#create corpus

texts=data_lemmatized

corpus=[idtoword.doc2bow(text) for text in texts]

In [95]:
len(corpus)

1254646

# Gensim LDA Model -Topic Modeling

In [96]:
#build our LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=idtoword,
                                           num_topics=5,
                                           workers=3,
                                           random_state=100,
                                           eval_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=1)


In [97]:
f=open("./model/lda_model_december.pkl",'wb')
pickle.dump(lda_model,f)
f.close()

In [98]:
f=open("./model/idtoword_december.pkl",'wb')
pickle.dump(idtoword,f)
f.close()

In [99]:
f=open("./model/corpus_december.pkl",'wb')
pickle.dump(corpus,f)
f.close()

In [89]:
#pickle.dump(lda_model, open("./model/lda.pickle", "wb"))
#pickle.dump(idtoword, open("./model/idatoword_final.pickle", "wb"))
#pickle.dump(corpus, open("./model/corpus_final.pickle", "wb"))


In [100]:
print(lda_model.print_topics())

[(0, '0.072*"apartment" + 0.065*"clean" + 0.061*"recommend" + 0.049*"comfortable" + 0.034*"well" + 0.032*"check" + 0.029*"space" + 0.025*"need" + 0.023*"host" + 0.023*"highly"'), (1, '0.039*"stay" + 0.033*"time" + 0.032*"host" + 0.030*"make" + 0.025*"feel" + 0.022*"thank" + 0.022*"home" + 0.019*"day" + 0.019*"come" + 0.017*"back"'), (2, '0.038*"room" + 0.017*"night" + 0.016*"bathroom" + 0.016*"bed" + 0.011*"small" + 0.010*"bedroom" + 0.010*"kitchen" + 0.009*"use" + 0.009*"get" + 0.009*"sleep"'), (3, '0.156*"place" + 0.155*"great" + 0.087*"stay" + 0.087*"location" + 0.060*"nice" + 0.053*"good" + 0.041*"host" + 0.040*"really" + 0.034*"clean" + 0.031*"super"'), (4, '0.046*"close" + 0.035*"walk" + 0.030*"subway" + 0.026*"neighborhood" + 0.026*"area" + 0.024*"restaurant" + 0.023*"minute" + 0.019*"station" + 0.019*"lot" + 0.018*"train"')]


In [101]:
lda_model.save('lda_december.model')

In [102]:
# Visualize the topics
pyLDAvis.enable_notebook(sort=True)
vis = pyLDAvis.gensim.prepare(lda_model, corpus, idtoword)
pyLDAvis.display(vis)

/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [161]:
pyLDAvis.save_html(vis, 'December_lda.html')

### TOPICS ARE
    0:Aparment Amenities
    1:Host Experience
    2:Location
    3:Host Communication
    4:Overall Experience

In [103]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))



Perplexity:  -6.533763486670208


In [104]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=idtoword, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:504: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:506: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:320: DeprecationWarning: Numpy has detected that you (may be) writing to an array with
overlapping memory from np.broadcast_arrays. If this is intentional
set the WRITEABLE flag True or make a copy immediately before writing.
  i, j, x)
/Users/ikhyvicky/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:320: DeprecationWarning: Numpy ha


Coherence Score:  0.6041837351012622


In [108]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [109]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)

In [110]:
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

In [111]:
df_dominant_topic.shape

(1254646, 5)

In [112]:
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2.0,0.4564,"room, night, bathroom, bed, small, bedroom, ki...",Our stay of three nights.\nWe appreciate The a...
1,1,3.0,0.2857,"place, great, stay, location, nice, good, host...",Great experience.
2,2,3.0,0.3274,"place, great, stay, location, nice, good, host...",I've stayed with my friend at the Midtown Cast...
3,3,3.0,0.3450,"place, great, stay, location, nice, good, host...","We've been staying here for about 9 nights, en..."
4,4,0.0,0.4837,"apartment, clean, recommend, comfortable, well...",We had a wonderful stay at Jennifer's charming...
5,5,0.0,0.2470,"apartment, clean, recommend, comfortable, well...",Hi to everyone!\nWould say our greatest compli...
6,6,3.0,0.3273,"place, great, stay, location, nice, good, host...","Jennifer was very friendly and helpful, and he..."
7,7,2.0,0.5383,"room, night, bathroom, bed, small, bedroom, ki...",This apartment is like a real castle old and u...
8,8,1.0,0.2633,"stay, time, host, make, feel, thank, home, day...",Jennifer's place was in a great midtown locati...
9,9,0.0,0.3455,"apartment, clean, recommend, comfortable, well...",Jennifer is a very nice host. Everything is cl...


In [113]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [114]:
sia = SentimentIntensityAnalyzer()

In [115]:
example=df_dominant_topic.loc[:10,:]

In [116]:
example.shape

(11, 5)

In [117]:
for sentence in example.Text:
    # Assess sentiment.
    sentiment = sia.polarity_scores(sentence)["compound"]
    print(sentence)
    print(sentiment)
    print('\n')

Our stay of three nights.
We appreciate The apartment is great. Nice, clean and well groomed. This is ideal for a family of 3 or 4 people.
Glitches arriving there was no one to receive us, and missing the shower soap, dishwashing liquid, we have received two days later. There is also the sound of the Metro NY, so the first night difficult if not used. Jennifer is the correct refund was very fast. Apart from these small details our short stay went well.
If I have the opportunity to return to NY for the holidays, I will return to "The Midtown Castle"
Jeans
Possession - Reunion Island

0.9545


Great experience.
0.6249


I've stayed with my friend at the Midtown Castle for six days and it was a lovely place to be. A big spacious room with a pointy roof, which really makes you feel like staying in a castle. The location is perfect. It is just a few steps from Macy's Time Square and Theatre District. Everything worked just perfect with the keys etc. Thank you so much Jennifer, we had a grea

In [118]:
sentiment_scores=[]
for sentence in df_dominant_topic.Text:
    sentiment = sia.polarity_scores(sentence)["compound"]
    sentiment_scores.append(sentiment)
    

In [119]:
df_dominant_topic['sentiment_score'] = pd.Series(sentiment_scores, index=df_dominant_topic.index)

In [120]:
df_dominant_topic.shape

(1254646, 6)

In [121]:
df_dominant_topic.head()

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,sentiment_score
0,0,2.0,0.4564,"room, night, bathroom, bed, small, bedroom, ki...",Our stay of three nights.\nWe appreciate The a...,0.9545
1,1,3.0,0.2857,"place, great, stay, location, nice, good, host...",Great experience.,0.6249
2,2,3.0,0.3274,"place, great, stay, location, nice, good, host...",I've stayed with my friend at the Midtown Cast...,0.9248
3,3,3.0,0.3450,"place, great, stay, location, nice, good, host...","We've been staying here for about 9 nights, en...",0.9229
4,4,0.0,0.4837,"apartment, clean, recommend, comfortable, well...",We had a wonderful stay at Jennifer's charming...,0.9464


In [122]:
#to get a sense of what negative comments will look like
negative_sentiments=df_dominant_topic[df_dominant_topic['sentiment_score']<0]

In [123]:
negative_sentiments.reset_index(drop=True,inplace=True)

In [124]:
negative_sentiments.shape

(21795, 6)

In [125]:
negative_sentiments.sort_values('sentiment_score').head(20)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,sentiment_score
6313,446162,2.0,0.6093,"room, night, bathroom, bed, small, bedroom, ki...",I’ve used Airbnb close to 30 times and this by...,-0.9980
20415,1207614,2.0,0.6894,"room, night, bathroom, bed, small, bedroom, ki...",I had the worse experience with AIRBNB for the...,-0.9973
7760,535392,2.0,0.5869,"room, night, bathroom, bed, small, bedroom, ki...",READ BEFORE BOOKING!!!!!!!!!!!!!!!!!!!!!!!!!!!...,-0.9969
9808,646203,2.0,0.6407,"room, night, bathroom, bed, small, bedroom, ki...",The worst mistake of our trip to NY // THE WOR...,-0.9967
26,753,2.0,0.6911,"room, night, bathroom, bed, small, bedroom, ki...",The host was present in the department every n...,-0.9966
2476,186753,2.0,0.7800,"room, night, bathroom, bed, small, bedroom, ki...",I stayed here for a month and commuted to Linc...,-0.9966
3699,270451,2.0,0.7805,"room, night, bathroom, bed, small, bedroom, ki...",OUR WORST EXPERIENCE WITH AIRBNB The apartment...,-0.9964
4920,350802,2.0,0.6418,"room, night, bathroom, bed, small, bedroom, ki...",Thanksgiving family vacation gone bad. Jin was...,-0.9964
2579,195597,2.0,0.6861,"room, night, bathroom, bed, small, bedroom, ki...",Shame there are listings such as this on Airbn...,-0.9963
4522,328225,2.0,0.6934,"room, night, bathroom, bed, small, bedroom, ki...",We stayed for three months at Williams and Ana...,-0.9963


In [129]:
negative_sentiments.Text.iloc[7760]

'READ BEFORE BOOKING!!!!!!!!!!!!!!!!!!!!!!!!!!!\n\nHello, I am so livid, and dont know who else can help me get some justice. These people need to be called out in a major way!!!! \n\n I was a resident of 702 44th St, BK, the site of the huge 6 alarm fire in Sunset Park on Wed April 4. My family and I are safe, but we lost everything. I have a 5 month old baby, and we evacuated the building with nothing but the clothes on our backs. \n\nI had my first day back from Maternity Leave on April 4, the day of the fire, and my mother is visiting to help with childcare, we booked this for her. After evacuating, my mother, newborn and I raced barefoot back to the AirBnB, while my husband stayed in the park with our 12 year old dog, Tilly. Tilly had a procedure at the vet Tues night and is currently wearing a cone, and on pain med.\nUpon frantically arriving to the AirBnB, we greeted our host, who was watching news coverage of the fire. We explained that it was my building and now my family, inc

In [127]:
negative_sentiments.Dominant_Topic.nunique() #this just means people  can talk negative things for each topic

5

In [130]:
#putting them back together
final_df.head()

,index,listing_id,id,date,reviewer_id,reviewer_name,comments,translated
0,0,2595,17857,2009-11-21,50679,Jean,Notre séjour de trois nuits.\r\nNous avons app...,Our stay of three nights.\nWe appreciate The a...
1,1,2595,19176,2009-12-05,53267,Cate,Great experience.,Great experience.
2,2,2595,19760,2009-12-10,38960,Anita,I've stayed with my friend at the Midtown Cast...,I've stayed with my friend at the Midtown Cast...
3,3,2595,34320,2010-04-09,71130,Kai-Uwe,"We've been staying here for about 9 nights, en...","We've been staying here for about 9 nights, en..."
4,4,2595,46312,2010-05-25,117113,Alicia,We had a wonderful stay at Jennifer's charming...,We had a wonderful stay at Jennifer's charming...


In [131]:
final_df.shape

(1254646, 8)

In [132]:
df_dominant_topic.shape

(1254646, 6)

In [133]:
df_dominant_topic.head()

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,sentiment_score
0,0,2.0,0.4564,"room, night, bathroom, bed, small, bedroom, ki...",Our stay of three nights.\nWe appreciate The a...,0.9545
1,1,3.0,0.2857,"place, great, stay, location, nice, good, host...",Great experience.,0.6249
2,2,3.0,0.3274,"place, great, stay, location, nice, good, host...",I've stayed with my friend at the Midtown Cast...,0.9248
3,3,3.0,0.3450,"place, great, stay, location, nice, good, host...","We've been staying here for about 9 nights, en...",0.9229
4,4,0.0,0.4837,"apartment, clean, recommend, comfortable, well...",We had a wonderful stay at Jennifer's charming...,0.9464


In [134]:
#drop original non-translated comments
final_df.drop(columns=['comments'],axis=1,inplace=True)

In [135]:
final_df.reset_index(inplace=True)

In [136]:
df_dominant_topic.drop(columns=['Document_No','Text'],axis=1,inplace=True)

In [137]:
#combine the two together 
frames=[final_df,df_dominant_topic]
finaldf=pd.concat(frames,axis=1)

In [151]:
finaldf.head()

,listing_id,id,date,reviewer_id,reviewer_name,translated,Dominant_Topic,Topic_Perc_Contrib,Keywords,sentiment_score
0,2595,17857,2009-11-21,50679,Jean,Our stay of three nights.\nWe appreciate The a...,2.0,0.4564,"room, night, bathroom, bed, small, bedroom, ki...",0.9545
1,2595,19176,2009-12-05,53267,Cate,Great experience.,3.0,0.2857,"place, great, stay, location, nice, good, host...",0.6249
2,2595,19760,2009-12-10,38960,Anita,I've stayed with my friend at the Midtown Cast...,3.0,0.3274,"place, great, stay, location, nice, good, host...",0.9248
3,2595,34320,2010-04-09,71130,Kai-Uwe,"We've been staying here for about 9 nights, en...",3.0,0.3450,"place, great, stay, location, nice, good, host...",0.9229
4,2595,46312,2010-05-25,117113,Alicia,We had a wonderful stay at Jennifer's charming...,0.0,0.4837,"apartment, clean, recommend, comfortable, well...",0.9464


In [141]:
finaldf.drop(columns=['level_0','index'],inplace=True)

In [142]:
finaldf.shape

(1254646, 10)

In [143]:
finaldf.isnull().sum()

listing_id            0
id                    0
date                  0
reviewer_id           0
reviewer_name         0
translated            0
Dominant_Topic        0
Topic_Perc_Contrib    0
Keywords              0
sentiment_score       0
dtype: int64

In [144]:
finaldf.to_csv('./final_december_df',index=False)

In [145]:
#now the tedious part is to link listing id to negative sentiment scores
final_negative_sentiments=finaldf[finaldf['sentiment_score']<0]

In [146]:
final_negative_sentiments.shape  #8996 comments with negative comments

(21795, 10)

In [147]:
final_negative_sentiments.listing_id.nunique() #10276 unique listings out of 40372 listings are negatively commented  25% of the listings

10276

In [158]:
finaldf.listing_id.nunique() 

40372

In [159]:
listings_sentiments=finaldf.groupby(['listing_id','Dominant_Topic']).agg({'id':'count', 'sentiment_score':'mean'})

In [160]:
listings_sentiments.head(20)

id  sentiment_score
listing_id Dominant_Topic                     
2595       0.0             10         0.890350
           1.0              8         0.959125
           2.0             13         0.692154
           3.0             14         0.814600
           4.0              3         0.492500
3831       0.0             44         0.835307
           1.0             66         0.874947
           2.0             86         0.719660
           3.0             52         0.834142
           4.0             47         0.866843
5099       0.0             12         0.896542
           1.0             26         0.688173
           2.0             16         0.701206
           3.0              7         0.926729
           4.0             17         0.877735
5121       0.0              2         0.838650
           1.0             28         0.910007
           2.0              4         0.655975
           3.0             10         0.880850
           4.0              5         0.952900

In [90]:
listings_sentiments.to_csv('./listings_sentiments.csv')